In [2]:
import os
import math
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.genmod.families.links as links
from statsmodels.stats.mediation import Mediation
from collections import defaultdict
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
sessions = pd.read_csv('sessions.csv')

The following code extracts the mean value of each combined EDA measures and computes the percentage of experimental time (in seconds) with JVA for mediation analysis. 

In [4]:
MD = defaultdict(list)

def getVars():
    
    directory = os.getcwd()
     
    for csv in os.listdir(directory):
        
        if "_combine_bySecond.csv" in csv:  
            
            groupNum = csv[:-21] #extract group number 
            df = pd.read_csv(csv, index_col = 'Unnamed: 0')
            
            MD['Session'].append(int(groupNum)) #append session number
            
            MD['DA'].append(np.mean(df['DA']))
            MD['SM'].append(np.mean(df['SM']))
            MD['PC'].append(np.mean(df['PC']))
            MD['IDM'].append(np.mean(df['IDM']))
            MD['jva100'].append(np.mean(df['jva100']))
            MD['JVA'].append(np.sum(df['JVA'])/max(df['second'])) #percentage of time with jva/avg JVA per second
            
    return MD

In [5]:
M = getVars()
dfM = pd.DataFrame.from_dict(M)
dfM.head()

,Session,DA,SM,PC,IDM,jva100,JVA
0,38,0.228346,0.700138,0.005122,0.282717,9.326601,0.645813
1,41,0.300937,1.248466,0.130912,0.153666,4.405405,0.459459
2,8,0.306824,1.133220,-0.019668,0.293786,13.269110,0.859686
3,2,0.320775,0.790207,-0.023976,0.324825,10.648069,0.768240
4,32,0.248145,0.816179,-0.040254,0.634861,9.578711,0.752624


In [8]:
mAnalysis = sessions.merge(dfM, on = 'Session', how = 'outer')
mAnalysis.head()

,Session,Session Date,Condition,Sustaining Mutual Understanding,Dialogue Management,Information Pooling,Reaching Consensus,Task Division,Time Managment,Technical Coordination,...,Task Understanding,Improvement Over Time,Code quality,Learning,DA,SM,PC,IDM,jva100,JVA
0,1,8-Aug,2,0.0,0.0,-1.0,-1.0,0.0,-2.0,-2.0,...,-1.0,0.0,NaN,NaN,0.345218,1.012636,0.328373,0.193013,4.472087,0.533333
1,2,10-Aug,2,1.0,1.0,-1.0,1.0,2.0,-1.0,-1.0,...,0.0,1.0,3.0,37.500000,0.320775,0.790207,-0.023976,0.324825,10.648069,0.768240
2,3,10-Aug,2,1.0,2.0,1.0,2.0,1.0,0.0,2.0,...,1.0,1.0,3.0,70.833333,NaN,NaN,NaN,NaN,NaN,NaN
3,4,11-Aug,1,2.0,2.0,1.0,2.0,0.0,0.0,0.0,...,1.0,NaN,3.0,66.666667,0.219271,1.086592,-0.128259,0.494100,4.490436,0.597237
4,5,11-Aug,1,-1.0,-1.0,1.0,0.0,-2.0,-1.0,0.0,...,1.0,NaN,2.0,20.833333,0.207562,1.120081,0.300314,0.566593,5.382572,0.564293


In [9]:
#rename columns
mAnalysis = mAnalysis.rename(columns = {"Sustaining Mutual Understanding ": 'Sustaining_Mutual_Understanding',
                                       'Dialogue Management':'Dialogue_Management', 'Information Pooling': 'Information_Pooling',
                                       'Reaching Consensus': 'Reaching_Consensus', 'Task Division': 'Task_Division',
                                       'Time Managment': 'Time_Management', 'Technical Coordination': 'Technical_Coordination',
                                       'Reciprocal Interaction': 'Reciprocal_Interaction', 'Individual Task Orientation': 'Individual_Task_Orientation',
                                       'Task Performance': 'Task_Performance','Task Understanding': 'Task_Understanding', 'Improvement Over Time': 'Improvement_Over_Time',
                                       'Code quality': 'Code_Quality'})

In [10]:
mAnalysis_dropna = mAnalysis.copy()
mAnalysis_dropna = mAnalysis_dropna.dropna() #drop nans

In [12]:
mAnalysis_dropna

,Session,Session Date,Condition,Sustaining_Mutual_Understanding,Dialogue_Management,Information_Pooling,Reaching_Consensus,Task_Division,Time_Management,Technical_Coordination,...,Task_Understanding,Improvement_Over_Time,Code_Quality,Learning,DA,SM,PC,IDM,jva100,JVA
1,2,10-Aug,2,1.0,1.0,-1.0,1.0,2.0,-1.0,-1.0,...,0.0,1.0,3.0,37.500000,0.320775,0.790207,-0.023976,0.324825,10.648069,0.768240
7,8,17-Aug,4,2.0,2.0,1.0,2.0,1.0,1.0,2.0,...,1.0,2.0,3.0,37.500000,0.306824,1.133220,-0.019668,0.293786,13.269110,0.859686
8,9,17-Aug,4,0.0,1.0,1.0,1.0,1.0,-2.0,-1.0,...,-1.0,-1.0,1.0,29.166667,0.270150,1.470201,-0.273257,0.440550,7.697300,0.705088
10,11,18-Aug,3,0.0,0.0,1.0,-1.0,-2.0,-1.0,0.0,...,1.0,0.0,2.0,100.000000,0.344646,0.602089,-0.017913,0.175545,13.161049,0.870519
11,12,18-Aug,1,1.0,1.0,1.0,1.0,0.0,-1.0,-1.0,...,-2.0,0.0,1.0,52.083333,0.255301,1.337541,-0.093416,0.474923,5.179583,0.559059
12,13,22-Aug,4,2.0,2.0,2.0,2.0,1.0,1.0,2.0,...,1.0,2.0,4.0,83.333333,0.341756,1.161427,-0.005435,0.332900,8.589958,0.632845
14,15,22-Aug,1,-2.0,-1.0,0.0,-1.0,-2.0,-1.0,0.0,...,1.0,1.0,3.0,31.250000,0.092475,1.180838,0.100167,0.077980,1.190200,0.171233
15,16,22-Aug,2,1.0,1.0,1.0,1.0,-2.0,0.0,0.0,...,-1.0,0.0,2.0,43.750000,0.197780,1.418616,-0.163536,0.324991,5.548471,0.711249
16,17,24-Aug,1,2.0,2.0,1.0,1.0,1.0,-2.0,-1.0,...,1.0,1.0,1.0,60.416667,0.270365,0.613347,-0.093778,0.220856,12.340151,0.817546
18,19,24-Aug,1,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,...,0.0,1.0,2.0,31.250000,0.290431,0.775878,0.092328,0.184361,9.170940,0.758547


In [11]:
dependent = ['Sustaining_Mutual_Understanding ', 'Dialogue_Management',
             'Information_Pooling', 'Reaching_Consensus', 'Task_Division',
             'Time_Management', 'Technical_Coordination', 'Reciprocal_Interaction',
             'Individual_Task_Orientation', 'Collaboration', 'Task_Performance',
             'Task_Understanding', 'Improvement_Over_Time', 'Code_Quality',
             'Learning']

**Mediations: JVA -> EDA -> Qualitative Outcome Measures**

Below I examined each qualitative EDA measure as the mediator of the relationship between percentage of time with JVA and each qualitative outcome measure. 

In [23]:
#DA as the mediator
outcome_models_da = list()

#append each mediation model to list
for vars in dependent:
    outcome_models_da.append(vars + '~ JVA + DA')
    
for mod in outcome_models_da:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("DA ~ JVA", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "JVA", "DA").fit() 
    print(med.summary())
    print("="*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + DA
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.093034       -0.194521        0.500725    0.532
ACME (treated)            0.093034       -0.194521        0.500725    0.532
ADE (control)             2.618898        1.358544        3.944304    0.000
ADE (treated)             2.618898        1.358544        3.944304    0.000
Total effect              2.711932        1.444601        4.037564    0.000
Prop. mediated (control)  0.024667       -0.074146        0.197028    0.532
Prop. mediated (treated)  0.024667       -0.074146        0.197028    0.532
ACME (average)            0.093034       -0.194521        0.500725    0.532
ADE (average)             2.618898        1.358544        3.944304    0.000
Prop. mediated (average)  0.024667       -0.074146        0.197028    0.532

Dialogue_Management~ JVA + DA
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.862255       -1.189035        3.967237    0.434
ACME (treated)            0.862255       -1.189035        3.967237    0.434
ADE (control)             7.636642       -0.811594       16.036329    0.080
ADE (treated)             7.636642       -0.811594       16.036329    0.080
Total effect              8.498898        0.188692       16.868706    0.046
Prop. mediated (control)  0.076208       -0.244707        0.853008    0.440
Prop. mediated (treated)  0.076208       -0.244707        0.853008    0.440
ACME (average)            0.862255       -1.189035        3.967237    0.434
ADE (average)             7.636642       -0.811594       16.036329    0.080
Prop. mediated (average)  0.076208       -0.244707        0.853008    0.440

Task_Performance~ JVA + DA
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.105855       -0.107818        0.

The above mediation analysis examines Directional Agreement as a mediator of the relationship between percentage of time with JVA and qualitative outcome measures. Highlights of results are as follows:

* Percentage of time with JVA significantly predicted 1) Sustaining Mutual Understanding, 2) Dialogue Management, 3) Reaching Consensus after controlling for Directional Agreement.

* There is no significant mediation effect observed. 

In [25]:
#SM as the mediator
outcome_models_sm = list()
for vars in dependent:
    outcome_models_sm.append(vars+'~ JVA + SM')
for mod in outcome_models_sm:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("SM ~ JVA", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "JVA", "SM").fit() #SM as mediator
    print(med.summary())
    print("="*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + SM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.094068       -0.210039        0.547337    0.556
ACME (treated)            0.094068       -0.210039        0.547337    0.556
ADE (control)             2.605970        1.274065        3.876082    0.000
ADE (treated)             2.605970        1.274065        3.876082    0.000
Total effect              2.700038        1.389708        3.945958    0.000
Prop. mediated (control)  0.023609       -0.100974        0.222156    0.556
Prop. mediated (treated)  0.023609       -0.100974        0.222156    0.556
ACME (average)            0.094068       -0.210039        0.547337    0.556
ADE (average)             2.605970        1.274065        3.876082    0.000
Prop. mediated (average)  0.023609       -0.100974        0.222156    0.556

Dialogue_Management~ JVA + SM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.076689       -2.435653        2.021269    0.950
ACME (treated)           -0.076689       -2.435653        2.021269    0.950
ADE (control)             8.525573        0.501110       17.061833    0.038
ADE (treated)             8.525573        0.501110       17.061833    0.038
Total effect              8.448884        0.460563       16.768619    0.038
Prop. mediated (control) -0.001735       -0.549857        0.495476    0.964
Prop. mediated (treated) -0.001735       -0.549857        0.495476    0.964
ACME (average)           -0.076689       -2.435653        2.021269    0.950
ADE (average)             8.525573        0.501110       17.061833    0.038
Prop. mediated (average) -0.001735       -0.549857        0.495476    0.964

Task_Performance~ JVA + SM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)             0.33986       -0.086579        0.

The above mediation analysis examines Signal Matching as a mediator of the relationship between percentage of time with JVA and qualitative outcome measures. Highlights of results are as follows:

* Percentage of time with JVA significantly predicted 1) Sustaining Mutual Understanding, 2) Dialogue Management, 3) Reaching Consensus, 3) Task Division, 4) Collaboration,  after controlling for Signal Matching.

* There is no significant mediation effect observed. 

In [26]:
#PC as mediator
outcome_models_pc = list()
for vars in dependent:
    outcome_models_pc.append(vars+'~ JVA + PC')
for mod in outcome_models_pc:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("PC ~ JVA", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "JVA", "PC").fit() #PC as mediator
    print(med.summary())
    print("="*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + PC
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.003320       -0.324909        0.321511    0.934
ACME (treated)           -0.003320       -0.324909        0.321511    0.934
ADE (control)             2.677259        1.451730        3.959350    0.000
ADE (treated)             2.677259        1.451730        3.959350    0.000
Total effect              2.673939        1.454916        3.919398    0.000
Prop. mediated (control) -0.001168       -0.153364        0.127236    0.934
Prop. mediated (treated) -0.001168       -0.153364        0.127236    0.934
ACME (average)           -0.003320       -0.324909        0.321511    0.934
ADE (average)             2.677259        1.451730        3.959350    0.000
Prop. mediated (average) -0.001168       -0.153364        0.127236    0.934

Dialogue_Management~ JVA + PC
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.346514       -1.641779        2.545575    0.716
ACME (treated)            0.346514       -1.641779        2.545575    0.716
ADE (control)             8.183604       -0.222252       16.491920    0.060
ADE (treated)             8.183604       -0.222252       16.491920    0.060
Total effect              8.530117        0.901539       16.375194    0.034
Prop. mediated (control)  0.025804       -0.332055        0.574455    0.726
Prop. mediated (treated)  0.025804       -0.332055        0.574455    0.726
ACME (average)            0.346514       -1.641779        2.545575    0.716
ADE (average)             8.183604       -0.222252       16.491920    0.060
Prop. mediated (average)  0.025804       -0.332055        0.574455    0.726

Task_Performance~ JVA + PC
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.181929       -0.568017        0.

The above mediation analysis examines Pearson's Correlation Coefficients as a mediator of the relationship between percentage of time with JVA and qualitative outcome measures. Highlights of results are as follows:

* Percentage of time with JVA significantly predicted 1) Sustaining Mutual Understanding, 2) Dialogue Management, 3) Reaching Consensus, 3) Task Division, after controlling for Pearson's Correlation Coefficients.

* Percentage of time with JVA significantly predicted Collaboration (Collaboration ~ JVA).

* There is no significant mediation effect observed. 

In [28]:
#IDM as mediator 
outcome_models_idm = list()
for vars in dependent:
    outcome_models_idm.append(vars+'~ JVA + IDM')
for mod in outcome_models_idm:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("IDM ~ JVA", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "JVA", "IDM").fit() #IDM as mediator
    print(med.summary())
    print('='*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + IDM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.112476       -0.160353        0.535749     0.49
ACME (treated)            0.112476       -0.160353        0.535749     0.49
ADE (control)             2.588506        1.365145        3.849487     0.00
ADE (treated)             2.588506        1.365145        3.849487     0.00
Total effect              2.700982        1.464185        4.030904     0.00
Prop. mediated (control)  0.025142       -0.068466        0.199521     0.49
Prop. mediated (treated)  0.025142       -0.068466        0.199521     0.49
ACME (average)            0.112476       -0.160353        0.535749     0.49
ADE (average)             2.588506        1.365145        3.849487     0.00
Prop. mediated (average)  0.025142       -0.068466        0.199521     0.49

Dialogue_Management~ JVA + IDM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (contro

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.937047       -1.434438        4.107057    0.426
ACME (treated)            0.937047       -1.434438        4.107057    0.426
ADE (control)             7.658200       -0.262421       15.975154    0.060
ADE (treated)             7.658200       -0.262421       15.975154    0.060
Total effect              8.595247       -0.068426       17.502557    0.052
Prop. mediated (control)  0.086341       -0.468754        0.643925    0.438
Prop. mediated (treated)  0.086341       -0.468754        0.643925    0.438
ACME (average)            0.937047       -1.434438        4.107057    0.426
ADE (average)             7.658200       -0.262421       15.975154    0.060
Prop. mediated (average)  0.086341       -0.468754        0.643925    0.438

Task_Performance~ JVA + IDM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.071649       -0.343641        0

The above mediation analysis examines Instantaneous Derivative Matching as a mediator of the relationship between percentage of time with JVA and qualitative outcome measures. Highlights of results are as follows:

* Percentage of time with JVA significantly predicted 1) Sustaining Mutual Understanding, 2) Dialogue Management, 3) Reaching Consensus, 3) Task Division, 4) Collaboration,  after controlling for Instantaneous Derivative Matching.

* There is no significant mediation effect observed. 

**Mediations: EDA -> JVA -> Qualitative Outcome Measures**

Below I examined percentage of time with JVA as the mediator of the relationship between each combined EDA measure and each qualitative outcome measure. 

In [29]:
for mod in outcome_models_da:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("JVA ~ DA", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "DA", "JVA").fit() 
    print(med.summary())
    print('='*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + DA
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            1.295064       -0.318470        3.272990    0.130
ACME (treated)            1.295064       -0.318470        3.272990    0.130
ADE (control)             1.190279       -1.819572        4.418172    0.476
ADE (treated)             1.190279       -1.819572        4.418172    0.476
Total effect              2.485344       -0.883640        5.958083    0.144
Prop. mediated (control)  0.455231       -1.923416        4.318290    0.210
Prop. mediated (treated)  0.455231       -1.923416        4.318290    0.210
ACME (average)            1.295064       -0.318470        3.272990    0.130
ADE (average)             1.190279       -1.819572        4.418172    0.476
Prop. mediated (average)  0.455231       -1.923416        4.318290    0.210

Dialogue_Management~ JVA + DA
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)

                           Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)             4.155201       -1.260788       12.435276    0.174
ACME (treated)             4.155201       -1.260788       12.435276    0.174
ADE (control)             10.362861      -10.281487       30.177130    0.288
ADE (treated)             10.362861      -10.281487       30.177130    0.288
Total effect              14.518062       -5.854235       34.785265    0.166
Prop. mediated (control)   0.220052       -1.046889        2.046104    0.272
Prop. mediated (treated)   0.220052       -1.046889        2.046104    0.272
ACME (average)             4.155201       -1.260788       12.435276    0.174
ADE (average)             10.362861      -10.281487       30.177130    0.288
Prop. mediated (average)   0.220052       -1.046889        2.046104    0.272

Task_Performance~ JVA + DA
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.197263       -0.89743

The above mediation analysis examines JVA as a mediator of the relationship between Directional Agreement and qualitative outcome measures. Highlights of results are as follows:

* Directional Agreement significantly predicted Task Division.

* There is no significant mediation or total effect observed. 

In [30]:
for mod in outcome_models_sm:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("JVA ~ SM", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "SM", "JVA").fit() 
    print(med.summary())
    print("="*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + SM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.317132       -0.845419        0.114510    0.144
ACME (treated)           -0.317132       -0.845419        0.114510    0.144
ADE (control)            -0.263477       -1.033396        0.450184    0.492
ADE (treated)            -0.263477       -1.033396        0.450184    0.492
Total effect             -0.580610       -1.415743        0.261621    0.160
Prop. mediated (control)  0.476946       -0.913885        5.498106    0.204
Prop. mediated (treated)  0.476946       -0.913885        5.498106    0.204
ACME (average)           -0.317132       -0.845419        0.114510    0.144
ADE (average)            -0.263477       -1.033396        0.450184    0.492
Prop. mediated (average)  0.476946       -0.913885        5.498106    0.204

Dialogue_Management~ JVA + SM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -1.024262       -3.205319        0.467817    0.216
ACME (treated)           -1.024262       -3.205319        0.467817    0.216
ADE (control)             0.269906       -4.683966        4.983883    0.922
ADE (treated)             0.269906       -4.683966        4.983883    0.922
Total effect             -0.754356       -5.927734        4.006674    0.780
Prop. mediated (control)  0.181867       -6.608582        7.034044    0.736
Prop. mediated (treated)  0.181867       -6.608582        7.034044    0.736
ACME (average)           -1.024262       -3.205319        0.467817    0.216
ADE (average)             0.269906       -4.683966        4.983883    0.922
Prop. mediated (average)  0.181867       -6.608582        7.034044    0.736

Task_Performance~ JVA + SM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.070938       -0.052263        0.

The above mediation analysis examines JVA as a mediator of the relationship between Signal Matching and qualitative outcome measures. Highlights of results are as follows:

* Signal Matching significantly predicted Time Management.

* Signal Matching significantly predicted 1) Task Performance, 2) Learning after controlling for JVA

* There is no significant mediation effect observed. 

In [31]:
for mod in outcome_models_pc:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("JVA ~ PC", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "PC", "JVA").fit() 
    print(med.summary())
    print('='*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + PC
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.777307       -2.058637        0.260261    0.148
ACME (treated)           -0.777307       -2.058637        0.260261    0.148
ADE (control)             0.055828       -1.756975        1.949840    0.954
ADE (treated)             0.055828       -1.756975        1.949840    0.954
Total effect             -0.721479       -2.842831        1.385161    0.528
Prop. mediated (control)  0.509410       -7.297226        7.721875    0.500
Prop. mediated (treated)  0.509410       -7.297226        7.721875    0.500
ACME (average)           -0.777307       -2.058637        0.260261    0.148
ADE (average)             0.055828       -1.756975        1.949840    0.954
Prop. mediated (average)  0.509410       -7.297226        7.721875    0.500

Dialogue_Management~ JVA + PC
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -2.488009       -7.689316        0.885825    0.188
ACME (treated)           -2.488009       -7.689316        0.885825    0.188
ADE (control)            -3.015116      -15.632175       10.565180    0.628
ADE (treated)            -3.015116      -15.632175       10.565180    0.628
Total effect             -5.503124      -18.316528        8.120838    0.382
Prop. mediated (control)  0.232508       -1.916943        4.346893    0.446
Prop. mediated (treated)  0.232508       -1.916943        4.346893    0.446
ACME (average)           -2.488009       -7.689316        0.885825    0.188
ADE (average)            -3.015116      -15.632175       10.565180    0.628
Prop. mediated (average)  0.232508       -1.916943        4.346893    0.446

Task_Performance~ JVA + PC
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.031490       -0.274613        0.

The above mediation analysis examines JVA as a mediator of the relationship between Pearson's Correlation Coefficients and qualitative outcome measures. Highlights of results are as follows:

* Pearson's Correlation Coefficients significantly predicted 1) Task Performance, 2) Code Quality after controlling for JVA

* There is no significant mediation effect observed. 

In [32]:
for mod in outcome_models_idm:
    print(mod)
    outcome_model = sm.OLS.from_formula(mod, mAnalysis_dropna)
    mediator_model = sm.OLS.from_formula("JVA ~ IDM", mAnalysis_dropna)
    med = Mediation(outcome_model, mediator_model, "IDM", "JVA").fit() 
    print(med.summary())
    print("="*50)
    print()

Sustaining_Mutual_Understanding ~ JVA + IDM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.191732       -0.244332        0.694182    0.400
ACME (treated)            0.191732       -0.244332        0.694182    0.400
ADE (control)             0.573361       -0.263165        1.335799    0.174
ADE (treated)             0.573361       -0.263165        1.335799    0.174
Total effect              0.765093       -0.178832        1.675379    0.120
Prop. mediated (control)  0.226784       -1.274978        1.996739    0.392
Prop. mediated (treated)  0.226784       -1.274978        1.996739    0.392
ACME (average)            0.191732       -0.244332        0.694182    0.400
ADE (average)             0.573361       -0.263165        1.335799    0.174
Prop. mediated (average)  0.226784       -1.274978        1.996739    0.392

Dialogue_Management~ JVA + IDM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (contro

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.590111       -0.997691        2.727911    0.474
ACME (treated)            0.590111       -0.997691        2.727911    0.474
ADE (control)             5.136923       -0.163113       10.195032    0.058
ADE (treated)             5.136923       -0.163113       10.195032    0.058
Total effect              5.727033        0.146262       11.005141    0.050
Prop. mediated (control)  0.077406       -0.389061        0.625849    0.484
Prop. mediated (treated)  0.077406       -0.389061        0.625849    0.484
ACME (average)            0.590111       -0.997691        2.727911    0.474
ADE (average)             5.136923       -0.163113       10.195032    0.058
Prop. mediated (average)  0.077406       -0.389061        0.625849    0.484

Task_Performance~ JVA + IDM
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.016806       -0.140931        0

The above mediation analysis examines JVA as a mediator of the relationship between Instantaneous Derivative Matching and qualitative outcome measures. Highlights of results are as follows:

* Instantaneous Derivative Matching significantly predicted 1) Reciprocal Interaction, 2) Code Quality after controlling for JVA

* There is no significant mediation effect observed. 